# Download glove embedding
 to represent word as vectors 

In [ ]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt


--2021-09-01 15:38:52--  https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53933315 (51M) [text/plain]
Saving to: ‘glove.840B.300d.sst.txt’

glove.840B.300d.sst 100%[===================>]  51.43M   194MB/s    in 0.3s    

2021-09-01 15:38:53 (194 MB/s) - ‘glove.840B.300d.sst.txt’ saved [53933315/53933315]



In [ ]:
!cp glove.840B.300d.sst.txt sample_data/glove.840B.300d.sst.txt

# Importing packages

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import itertools
from collections import Counter
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import *
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import array
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import sequence
from keras.regularizers import *
import tensorflow as tf
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
import timeit

# Supporting functions

In [ ]:
def dumpPickle(path,object_to_dump):
	f=open(path,'wb')
	pickle.dump(object_to_dump,f)
	f.close()

#Reads an object stores at 'path'
def readPickle(path):
	f=open(path,'rb')
	ob_to_load=pickle.load(f)
	f.close()
	return ob_to_load

#Loads an embedding matrix stores at emb_pickle_path if doesn't exists a pickled file, then creates One which is initialized by normal distribution with mean and std dev of the embeddings used, if given the option to save it saves
#the newly created embedding matrix at saveName
def load_create_embedding_matrix(word_index,vocab_size,emb_dim,emb_path,emb_pickle_path=False,save=False,saveName=None):
	if not emb_pickle_path:
		embedding_dict={}
		f=open(emb_path,'rb')
		for line in f:
			fields=line.split()
			word=fields[0]
			w_e=np.asarray(fields[1:],dtype='float32')
			embedding_dict[word]=w_e
		f.close()
		allembs=np.stack(embedding_dict.values())
		emb_mean,emb_std=allembs.mean(),allembs.std()
		embedding_matrix=np.random.normal(emb_mean,emb_std,(vocab_size,emb_dim))
		for word,index in word_index.items():
			#print(word)
			vector=embedding_dict.get(word)
			if vector is not None:
				embedding_matrix[index]=vector
		if save:
			dumpPickle(saveName,embedding_matrix)
	else:
		f=open(emb_pickle_path,'rb')
		embedding_matrix=pickle.load(f)
		f.close()
	return embedding_matrix


#creates a tokenizer from training data file in csv format, if there is one it loads and returns
def load_create_tokenizer(train_data,tok_path=None,savetokenizer=False):
	if tok_path == None:
		tokenizer=Tokenizer()
		tokenizer.fit_on_texts(train_data)
		len(tokenizer.word_index)
		if savetokenizer:
			dumpPickle('./New_Tokenizer.tkn',tokenizer)
	else:
		tokenizer=readPickle(tok_path)
	return tokenizer

def load_create_padded_data(X_train,savetokenizer,padPath=None,isPaddingDone=False,maxlen=None,tokenizer_path=None,save_new_padded_data=False,path_for_new_data=False):
	#print(tokenizer_path)
	if not isPaddingDone:
		tokenizer=load_create_tokenizer(X_train,tok_path=tokenizer_path,savetokenizer=savetokenizer)
		#word_index=tokenizer.word_index
		X_train=tokenizer.texts_to_sequences(X_train)
		X_train=pad_sequences(X_train,maxlen=maxlen)
		if save_new_padded_data:
			dumpPickle(path_for_new_data,X_train)
	else:
		X_train=readPickle(padPath)
	return X_train

# Loading Dataset

In [ ]:
data = pd.read_csv('/content/TWEET_DATA_IEEE_TCSS_-_Sheet1.csv') 
data.head()                  

,RT @Bipartisanism: Katie Couric just schooled #TedCruz on gay marriage in epic fashion. http://t.co/JtNOPqqOug,STM
0,RT @stevesilberman: If you missed it: Sick bur...,EXP
1,@jk_rowling & the Never-Ending Story: With a #...,QUE
2,RT @somebadideas: The emotional & personal eff...,EXP
3,RT @jk_rowling: A 9-year-old Nigerian girl has...,STM
4,RT @jk_rowling: I don't want to say too much m...,EXP


## Spliting in the ratio of 80-20 for test and train

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, test_size=0.2) 
train=pd.DataFrame(train)
train.columns=["Tweet","Label"]
test=pd.DataFrame(test)
test.columns=["Tweet","Label"]

In [ ]:
train_Text=train["Tweet"]
train_label=train["Label"]
test_Text=test["Tweet"]
test_label=test["Label"]

train = [s.strip() for s in train_Text]
test = [s.strip() for s in test_Text]

text = train + test
max1=0

#making same length of all text 
for i in range(0,len(text)):
	data=text[i].split(" ")
	max2=len(data)
	if(max2>max1):
		max1=max2


sequence_length = max1

# Create padded data and embedding matrix

In [ ]:
print("creating data")


tokenizer=load_create_tokenizer(train,None,True)
X_train=load_create_padded_data(X_train=train,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
X_test=load_create_padded_data(X_train=test,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
word_index=tokenizer.word_index
embedding_matrix=load_create_embedding_matrix(word_index,len(word_index)+1,300,'./glove.840B.300d.sst.txt',False,True,'./Emb_Mat.mat')

print("data created")

creating data


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


data created


# one-hot encode

In [ ]:
lbl_dict={}
index=0
for dial_lbls in train_label:
	if dial_lbls not in lbl_dict:
		lbl_dict[dial_lbls]=index
		index=index+1
print((lbl_dict))

def create_label(label):
	
    Y=[]
    for i in label:
    	xxx=np.zeros(len(lbl_dict)) 
    	j=lbl_dict.get(i)
    	xxx[j]=1
    	Y.append(xxx)
    return Y

label = train_label
Y_train = create_label(label)

label = test_label
Y_test = create_label(label)

y_train=np.array([i for i in Y_train])
y_test=np.array([i for i in Y_test])

embedding_dim = 300

{'THT': 0, 'EXP': 1, 'STM': 2, 'QUE': 3, 'OTH': 4, 'SUG': 5, 'REQ': 6}


In [ ]:
X_train.shape

(5999, 57)

In [ ]:
embedding_matrix.shape

(15263, 300)

In [ ]:
X_test.shape

(1500, 57)

# Creating CNN 1D model 

In [ ]:

print("Creating Model...")
#input layer
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=len(word_index)+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=sequence_length)(inputs)
conv_0 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(embedding)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)
dropout = Dropout(0.1)(maxpool_0)
conv_1 = Conv1D(filters=32, kernel_size=3, padding='same', kernel_initializer='normal', activation='relu')(dropout)
maxpool_1 = MaxPooling1D(pool_size=2)(conv_1)
flatten = Flatten()(maxpool_1)
predictions = Dense(1000, activation='relu')(flatten)
#predictions1 = output layer
#Lets assume, correct output => [0,1,0,0,0,0,0,0]; generated output => [0.03, 0.90, 0.01, 0.02, 0.01, 0.01, 0.01, 0.01]
predictions1 = Dense(len(lbl_dict), activation='softmax')(predictions)
adam = Adam(lr=0.01, decay=0.3)
model = Model(inputs=inputs, outputs=predictions1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_test, y_test))


predicted=model.predict(X_test)
#print("DONE")


Creating Model...
Epoch 1/4


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


94/94 [==============================] - 9s 88ms/step - loss: 1.4789 - accuracy: 0.5086 - val_loss: 1.3100 - val_accuracy: 0.5253
Epoch 2/4
94/94 [==============================] - 8s 86ms/step - loss: 1.2487 - accuracy: 0.5401 - val_loss: 1.2564 - val_accuracy: 0.5453
Epoch 3/4
94/94 [==============================] - 8s 84ms/step - loss: 1.1841 - accuracy: 0.5753 - val_loss: 1.2187 - val_accuracy: 0.5647
Epoch 4/4
94/94 [==============================] - 8s 84ms/step - loss: 1.1328 - accuracy: 0.5951 - val_loss: 1.1863 - val_accuracy: 0.5820


In [ ]:
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 57)]              0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 57, 300)           4578900   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 57, 32)            28832     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 28, 32)            0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 28, 32)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 28, 32)            3104      
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 14, 32)            0  

# Evaluate CNN model

In [ ]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_predict
# from sklearn.model_selection import train_test_split

print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.582
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        55
           1     0.5833    0.9556    0.7244       788
           2     0.5742    0.3550    0.4388       338
           3     0.0000    0.0000    0.0000       125
           4     0.0000    0.0000    0.0000        75
           5     0.0000    0.0000    0.0000        73
           6     0.0000    0.0000    0.0000        46

    accuracy                         0.5820      1500
   macro avg     0.1653    0.1872    0.1662      1500
weighted avg     0.4358    0.5820    0.4794      1500

[[  0  21  34   0   0   0   0]
 [  0 753  35   0   0   0   0]
 [  0 218 120   0   0   0   0]
 [  0 118   7   0   0   0   0]
 [  0  71   4   0   0   0   0]
 [  0  68   5   0   0   0   0]
 [  0  42   4   0   0   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Bi-LSTM model 

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  
maxlen=200
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 72)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_13 (Embedding)     (None, None, 72)          1440000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, None, 128)         70144     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 129       
Total params: 1,609,089
Trainable params: 1,609,089
Non-trainable params: 0
_________________________________________________________________


# evalute Bi-LSTM model

In [ ]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
    
    
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_predict
# from sklearn.model_selection import train_test_split

print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.582
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        55
           1     0.5833    0.9556    0.7244       788
           2     0.5742    0.3550    0.4388       338
           3     0.0000    0.0000    0.0000       125
           4     0.0000    0.0000    0.0000        75
           5     0.0000    0.0000    0.0000        73
           6     0.0000    0.0000    0.0000        46

    accuracy                         0.5820      1500
   macro avg     0.1653    0.1872    0.1662      1500
weighted avg     0.4358    0.5820    0.4794      1500

[[  0  21  34   0   0   0   0]
 [  0 753  35   0   0   0   0]
 [  0 218 120   0   0   0   0]
 [  0 118   7   0   0   0   0]
 [  0  71   4   0   0   0   0]
 [  0  68   5   0   0   0   0]
 [  0  42   4   0   0   0   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Few incorrect samples from the test set and deduce the possible reasons for mis-classification

It was a big data set tough, the sample from test case that I feel was wrong or mis-classified are 

*   RT @afterglow2046: Jane Austen or Charlotte Bronte?  Oh come on. That's like saying Air or water? 20 Questions with Ursula K Le Guin https
    > CORRECT - QUE 
    || GIVEN -   STM





*    RT @neemapr: #yesallwomen bc quickest way to get rid of unwanted attn is: 'I have a boyfriend.' 
    >CORRECT - SUG ||
    GIVEN -EXP







*  RT @lolligagme: #YesAllWomen Because when I drove cross country by myself I was never scared, but most women I know get nervous in a dark place
    >CORRECT - STM ||
    GIVEN - QUE


REASON -


1. **Sarcasm** 

Going through the data , I realised there were many sarcastic tweets available . Here words and phrases totally gets a different meaning . which becomes a challenge to idetify.


2. **use of synonyms** 

Synonyms can lead to issues similar to contextual understanding because we use many different words to express the same idea. 


3. **slang** 

Informal phrases, expressions, idioms, and culture-specific lingo present in various tweets which also changes the meaning of sentence.

4. **Ambiguity**

Even for humans this sentence alone is difficult to interpret without the context of surrounding text. simple example asking a question without using key words like "when" "how" "where" etc   













